<a href="https://colab.research.google.com/github/romellfudi/medium/blob/main/Pareto_Front_Emerging_Patterns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Jupyter Init - Python Colab
!pip install watermark -q
%load_ext watermark
from IPython.display import clear_output 
clear_output()
%watermark -a "Romell D.Z. Software Engineer and Business Intelligence Analyst" -gu romellfudi -iv -u -d -m

Author: Romell D.Z. Software Engineer and Business Intelligence Analyst

Github username: romellfudi

Last updated: 2022-12-16

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.10.133+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit

numpy     : 1.21.6
scikitplot: 0.3.7
matplotlib: 3.2.2
pandas    : 1.3.5
plotly    : 5.5.0



## Import libraries

In [18]:
!pip install -q scikit-plot pyfim paretoset hypernetx
from IPython.display import clear_output 
import pandas as pd
import numpy as np
import scikitplot as skplt
import matplotlib.pyplot as plt 
from paretoset import paretoset
from fim import *
import plotly.express as px 
clear_output()

# Emerging Patterns

In [ ]:
df_siembra = pd.read_excel('https://www.datosabiertos.gob.pe/sites/default/files/Estadisticas%20Intension%20de%20Siembra.xlsx')
df_siembra.rename(columns={'PROVINICA':'PROVINCIA'},inplace=True)
df_siembra_freq = df_siembra.copy()
df_siembra_freq['DISTRITO'] = df_siembra_freq['DEPARTAMENTO']+'-'+ df_siembra_freq['PROVINCIA']+'-'+ df_siembra_freq['DISTRITO']
df_siembra_freq

,DEPARTAMENTO,PROVINCIA,DISTRITO,CULTIVO,CAMPANA,AGO,SEP,OCT,NOV,DIC,ENE,FEB,MAR,ABR,MAY,JUN,JUL
0,ANCASH,AIJA,ANCASH-AIJA-SUCCHA,Papa nativa,2020-2021,0,0,1,0,0,0,0,0,0,0,0,0
1,ANCASH,ANTONIO RAYMONDI,ANCASH-ANTONIO RAYMONDI-LLAMELLIN,Olluco,2020-2021,0,0,1,0,0,0,0,0,0,0,0,0
2,ANCASH,ANTONIO RAYMONDI,ANCASH-ANTONIO RAYMONDI-LLAMELLIN,Quinua,2020-2021,0,0,1,0,0,0,0,0,0,0,0,0
3,ANCASH,ANTONIO RAYMONDI,ANCASH-ANTONIO RAYMONDI-ACZO,Quinua,2020-2021,0,0,1,0,0,0,0,0,0,0,0,0
4,ANCASH,ANTONIO RAYMONDI,ANCASH-ANTONIO RAYMONDI-CHINGAS,Olluco,2020-2021,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12176,ANCASH,RECUAY,ANCASH-RECUAY-TAPACOCHA,Cebada grano,2020-2021,0,0,0,0,0,4,5,0,0,0,0,0
12177,ANCASH,RECUAY,ANCASH-RECUAY-TAPACOCHA,Haba grano seco,2020-2021,0,0,0,0,0,3,4,0,0,0,0,0
12178,ANCASH,RECUAY,ANCASH-RECUAY-TAPACOCHA,Trigo,2020-2021,0,0,0,0,0,3,5,0,0,0,0,0
12179,ANCASH,SANTA,ANCASH-SANTA-MACATE,Haba grano seco,2020-2021,0,0,0,0,0,3,2,0,0,0,0,0


In [ ]:
df_siembra_freq = df_siembra_freq[['DISTRITO','CULTIVO']]
df_siembra_freq.drop_duplicates(inplace=True)
df_siembra_freq.shape

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


(12181, 2)

In [ ]:
def to_transactional(df,column_trans,columns_items):
  transactions = []
  for v in df[column_trans].unique():
    transactions.append(list(df[df[column_trans] == v][columns_items].values))
  return transactions

trans = to_transactional(df_siembra_freq, 'DISTRITO', 'CULTIVO')
print(len(trans))

1503


In [ ]:
def all_itemsets(trans_, supp_=-1,target_='a'):
  r = fpgrowth(trans_, target=target_, supp=supp_, report='aS')
  df_items = pd.DataFrame(r)
  df_items.columns = ['Items','Freq','Freq(%)']
  df_items.sort_values(by='Freq',ascending=False,inplace=True) 
  df_items['Itemset'] = [str(sorted(x)) for x in df_items['Items'].tolist()]
  return df_items

df_siembra_cajamarca = df_siembra[df_siembra['DEPARTAMENTO'] == 'CAJAMARCA']
trans_cajamarca = to_transactional(df_siembra_cajamarca, 'DISTRITO','CULTIVO')
print(len(trans_cajamarca))
df_siembra_not_cajamarca = df_siembra[df_siembra['DEPARTAMENTO'] != 'CAJAMARCA']
trans_not_cajamarca = to_transactional(df_siembra_not_cajamarca, 'DISTRITO','CULTIVO')
print(len(trans_not_cajamarca))

126
1291


In [ ]:
df_c_itemsets = all_itemsets(trans_cajamarca,-1,'c')
df_c_itemsets

,Items,Freq,Freq(%),Itemset
2304,"(Frijol grano seco,)",116,92.063492,['Frijol grano seco']
2305,"(Maiz amilaceo,)",114,90.476190,['Maiz amilaceo']
2302,"(Papa blanca,)",109,86.507937,['Papa blanca']
2301,"(Papa blanca, Maiz amilaceo)",108,85.714286,"['Maiz amilaceo', 'Papa blanca']"
2298,"(Arveja grano seco,)",107,84.920635,['Arveja grano seco']
...,...,...,...,...
74,"(Zanahoria, Zapallo, Camote, Haba grano seco, ...",1,0.793651,"['Arveja grano seco', 'Arveja grano verde', 'C..."
62,"(Zanahoria, Zapallo, Papa nativa, Camote, Arve...",1,0.793651,"['Arveja grano seco', 'Arveja grano verde', 'C..."
54,"(Zanahoria, Ajo, Zapallo, Arroz cascara, Olluc...",1,0.793651,"['Ajo', 'Arroz cascara', 'Arveja grano seco', ..."
51,"(Zanahoria, Quinua, Maiz choclo, Olluco, Haba ...",1,0.793651,"['Arveja grano verde', 'Cebada grano', 'Haba g..."


In [ ]:
df_nc_itemsets = all_itemsets(trans_not_cajamarca,-1,'c')
df_nc_itemsets

,Items,Freq,Freq(%),Itemset
47739,"(Maiz amilaceo,)",776,60.108443,['Maiz amilaceo']
47738,"(Cebada grano,)",685,53.059644,['Cebada grano']
47736,"(Trigo,)",643,49.806352,['Trigo']
47732,"(Haba grano seco,)",638,49.419055,['Haba grano seco']
47708,"(Papa color,)",638,49.419055,['Papa color']
...,...,...,...,...
2251,"(Aji, Arroz cascara, Tomate, Zapallo, Yuca, Ma...",1,0.077459,"['Aji', 'Arroz cascara', 'Arveja grano seco', ..."
2063,"(Aji, Ajo, Cebolla cabeza roja, Zapallo, Papa ...",1,0.077459,"['Aji', 'Ajo', 'Cebolla cabeza roja', 'Maiz am..."
1945,"(Aji, Ajo, Tomate, Zapallo, Maiz amarillo duro...",1,0.077459,"['Aji', 'Ajo', 'Arveja grano seco', 'Arveja gr..."
10481,"(Tomate, Zanahoria, Camote, Zapallo, Yuca, Mai...",1,0.077459,"['Arveja grano seco', 'Arveja grano verde', 'C..."


In [ ]:
pd.set_option('display.max_colwidth',None)
emerging = df_c_itemsets.join(df_nc_itemsets.set_index('Itemset'),
                              on='Itemset',lsuffix='_c',rsuffix='_nc',how='outer').fillna(0)
emerging['GrowthRate_c'] = (emerging['Freq(%)_c'] / emerging['Freq(%)_nc'])
emerging.sort_values(by='GrowthRate_c',ascending=False,inplace=True)
emerging.reset_index(inplace=True,drop=True)
emerging['Freq'] = emerging['Freq_c'] + emerging['Freq_nc']
l = []
for i in range(emerging.shape[0]):
  if emerging.iloc[i,0] != 0:
    l += [emerging.iloc[i,0]]
  elif emerging.iloc[i,4] != 0:
    l += [emerging.iloc[i,4]]
emerging['Items_c'] = l
emerging['Size'] = [len(x) for x in emerging['Items_c'].tolist()]
emerging

,Items_c,Freq_c,Freq(%)_c,Itemset,Items_nc,Freq_nc,Freq(%)_nc,GrowthRate_c,Freq,Size
0,"(Ajo, Papa color, Olluco, Maiz choclo, Trigo, Papa blanca, Maiz amilaceo, Cebada grano, Arveja grano verde, Arveja grano seco, Frijol grano seco)",12.0,9.523810,"['Ajo', 'Arveja grano seco', 'Arveja grano verde', 'Cebada grano', 'Frijol grano seco', 'Maiz amilaceo', 'Maiz choclo', 'Olluco', 'Papa blanca', 'Papa color', 'Trigo']",0,0.0,0.000000,inf,12.0,11
1,"(Papa nativa, Olluco, Yuca, Haba grano seco, Maiz choclo, Maiz amilaceo, Arveja grano verde, Papa blanca, Arveja grano seco)",9.0,7.142857,"['Arveja grano seco', 'Arveja grano verde', 'Haba grano seco', 'Maiz amilaceo', 'Maiz choclo', 'Olluco', 'Papa blanca', 'Papa nativa', 'Yuca']",0,0.0,0.000000,inf,9.0,9
2,"(Camote, Papa color, Olluco, Yuca, Maiz choclo, Maiz amarillo duro, Trigo, Arveja grano verde, Arveja grano seco, Papa blanca, Maiz amilaceo, Cebada grano, Frijol grano seco)",5.0,3.968254,"['Arveja grano seco', 'Arveja grano verde', 'Camote', 'Cebada grano', 'Frijol grano seco', 'Maiz amarillo duro', 'Maiz amilaceo', 'Maiz choclo', 'Olluco', 'Papa blanca', 'Papa color', 'Trigo', 'Yuca']",0,0.0,0.000000,inf,5.0,13
3,"(Arroz cascara, Camote, Yuca, Maiz choclo, Cebada grano, Maiz amarillo duro, Frijol grano seco, Maiz amilaceo, Papa blanca, Arveja grano seco, Arveja grano verde, Trigo)",5.0,3.968254,"['Arroz cascara', 'Arveja grano seco', 'Arveja grano verde', 'Camote', 'Cebada grano', 'Frijol grano seco', 'Maiz amarillo duro', 'Maiz amilaceo', 'Maiz choclo', 'Papa blanca', 'Trigo', 'Yuca']",0,0.0,0.000000,inf,5.0,12
4,"(Zapallo, Arroz cascara, Maiz choclo, Frijol grano seco, Maiz amilaceo, Yuca, Maiz amarillo duro, Papa blanca, Arveja grano seco, Arveja grano verde, Trigo, Cebada grano)",5.0,3.968254,"['Arroz cascara', 'Arveja grano seco', 'Arveja grano verde', 'Cebada grano', 'Frijol grano seco', 'Maiz amarillo duro', 'Maiz amilaceo', 'Maiz choclo', 'Papa blanca', 'Trigo', 'Yuca', 'Zapallo']",0,0.0,0.000000,inf,5.0,12
...,...,...,...,...,...,...,...,...,...,...
48297,"(Zanahoria, Quinua, Frijol grano seco, Papa nativa, Olluco)",0.0,0.000000,"['Frijol grano seco', 'Olluco', 'Papa nativa', 'Quinua', 'Zanahoria']","(Zanahoria, Quinua, Frijol grano seco, Papa nativa, Olluco)",34.0,2.633617,0.0,34.0,5
48298,"(Zanahoria, Arveja grano seco, Frijol grano seco, Olluco, Papa color, Papa blanca, Trigo, Cebada grano, Maiz amilaceo, Haba grano seco)",0.0,0.000000,"['Arveja grano seco', 'Cebada grano', 'Frijol grano seco', 'Haba grano seco', 'Maiz amilaceo', 'Olluco', 'Papa blanca', 'Papa color', 'Trigo', 'Zanahoria']","(Zanahoria, Arveja grano seco, Frijol grano seco, Olluco, Papa color, Papa blanca, Trigo, Cebada grano, Maiz amilaceo, Haba grano seco)",34.0,2.633617,0.0,34.0,10
48299,"(Zapallo, Maiz choclo, Quinua, Frijol grano seco, Papa nativa, Haba grano seco, Maiz amilaceo)",0.0,0.000000,"['Frijol grano seco', 'Haba grano seco', 'Maiz amilaceo', 'Maiz choclo', 'Papa nativa', 'Quinua', 'Zapallo']","(Zapallo, Maiz choclo, Quinua, Frijol grano seco, Papa nativa, Haba grano seco, Maiz amilaceo)",34.0,2.633617,0.0,34.0,7
48300,"(Zanahoria, Zapallo, Arveja grano verde, Papa color, Papa blanca)",0.0,0.000000,"['Arveja grano verde', 'Papa blanca', 'Papa color', 'Zanahoria', 'Zapallo']","(Zanahoria, Zapallo, Arveja grano verde, Papa color, Papa blanca)",34.0,2.633617,0.0,34.0,5


In [ ]:
mask = paretoset(emerging[['Freq','Size','GrowthRate_c']], sense=['max','max','max'])
sky_itemsets = emerging[mask]
emerging.loc[:,'is_skyitem'] = 'no'
sky_itemsets.loc[:,'is_skyitem'] = 'yes'
clear_output()
print(emerging.shape)
print(sky_itemsets.shape)
df = pd.concat([emerging,sky_itemsets]).reset_index()
fig = px.scatter_3d(df, x='Freq', y='Size', z='GrowthRate_c',color='is_skyitem')
# camera = dict( eye=dict(x=2, y=2, z=0.1) )
# fig.update_layout(scene_camera=camera,)
fig.show()

(48302, 11)
(169, 11)


In [ ]:
fig = px.parallel_coordinates(df[['Freq','Size','GrowthRate_c']] , color='Size',
                              labels={'Freq','Size','GrowthRate_c'},
                              color_continuous_scale = px.colors.diverging.Tealrose)
fig.show()

In [ ]:
df[df["GrowthRate_c"] < np.inf].sort_values(by="GrowthRate_c",ascending=False).head(7)

,index,Items_c,Freq_c,Freq(%)_c,Itemset,Items_nc,Freq_nc,Freq(%)_nc,GrowthRate_c,Freq,Size,is_skyitem
562,562,"(Arroz cascara, Yuca, Arveja grano verde, Maiz amarillo duro, Frijol grano seco)",20.0,15.873016,"['Arroz cascara', 'Arveja grano verde', 'Frijol grano seco', 'Maiz amarillo duro', 'Yuca']","(Arroz cascara, Yuca, Maiz amarillo duro, Arveja grano verde, Frijol grano seco)",11.0,0.852053,18.629149,31.0,5,no
563,563,"(Arroz cascara, Yuca, Arveja grano verde, Maiz amilaceo, Maiz amarillo duro, Frijol grano seco)",18.0,14.285714,"['Arroz cascara', 'Arveja grano verde', 'Frijol grano seco', 'Maiz amarillo duro', 'Maiz amilaceo', 'Yuca']","(Arroz cascara, Yuca, Maiz amarillo duro, Arveja grano verde, Maiz amilaceo, Frijol grano seco)",10.0,0.774593,18.442857,28.0,6,no
564,564,"(Yuca, Maiz choclo, Arveja grano verde, Papa blanca, Frijol grano seco, Arveja grano seco, Maiz amilaceo)",46.0,36.507937,"['Arveja grano seco', 'Arveja grano verde', 'Frijol grano seco', 'Maiz amilaceo', 'Maiz choclo', 'Papa blanca', 'Yuca']","(Yuca, Maiz choclo, Arveja grano verde, Arveja grano seco, Papa blanca, Frijol grano seco, Maiz amilaceo)",29.0,2.246321,16.252326,75.0,7,no
48318,564,"(Yuca, Maiz choclo, Arveja grano verde, Papa blanca, Frijol grano seco, Arveja grano seco, Maiz amilaceo)",46.0,36.507937,"['Arveja grano seco', 'Arveja grano verde', 'Frijol grano seco', 'Maiz amilaceo', 'Maiz choclo', 'Papa blanca', 'Yuca']","(Yuca, Maiz choclo, Arveja grano verde, Arveja grano seco, Papa blanca, Frijol grano seco, Maiz amilaceo)",29.0,2.246321,16.252326,75.0,7,yes
565,565,"(Arroz cascara, Cebada grano, Arveja grano seco, Maiz amarillo duro, Trigo, Papa blanca, Frijol grano seco, Maiz amilaceo)",17.0,13.492063,"['Arroz cascara', 'Arveja grano seco', 'Cebada grano', 'Frijol grano seco', 'Maiz amarillo duro', 'Maiz amilaceo', 'Papa blanca', 'Trigo']","(Arroz cascara, Maiz amarillo duro, Arveja grano seco, Frijol grano seco, Papa blanca, Trigo, Cebada grano, Maiz amilaceo)",11.0,0.852053,15.834776,28.0,8,no
48319,566,"(Yuca, Maiz choclo, Arveja grano verde, Frijol grano seco, Arveja grano seco, Maiz amilaceo)",49.0,38.888889,"['Arveja grano seco', 'Arveja grano verde', 'Frijol grano seco', 'Maiz amilaceo', 'Maiz choclo', 'Yuca']","(Yuca, Maiz choclo, Arveja grano verde, Arveja grano seco, Frijol grano seco, Maiz amilaceo)",32.0,2.478699,15.689236,81.0,6,yes
566,566,"(Yuca, Maiz choclo, Arveja grano verde, Frijol grano seco, Arveja grano seco, Maiz amilaceo)",49.0,38.888889,"['Arveja grano seco', 'Arveja grano verde', 'Frijol grano seco', 'Maiz amilaceo', 'Maiz choclo', 'Yuca']","(Yuca, Maiz choclo, Arveja grano verde, Arveja grano seco, Frijol grano seco, Maiz amilaceo)",32.0,2.478699,15.689236,81.0,6,no


In [ ]:
# Jumping Emerging Patterns (JEP) for Cajamarca Department
jep_c = df[df["GrowthRate_c"] == np.inf]
jep_c.sort_values(by="Freq_c", ascending=False,inplace=True)
jep_c

,index,Items_c,Freq_c,Freq(%)_c,Itemset,Items_nc,Freq_nc,Freq(%)_nc,GrowthRate_c,Freq,Size,is_skyitem
48311,434,"(Arveja grano verde, Arveja grano seco, Frijol grano seco)",81.0,64.285714,"['Arveja grano seco', 'Arveja grano verde', 'Frijol grano seco']",0,0.0,0.0,inf,81.0,3,yes
434,434,"(Arveja grano verde, Arveja grano seco, Frijol grano seco)",81.0,64.285714,"['Arveja grano seco', 'Arveja grano verde', 'Frijol grano seco']",0,0.0,0.0,inf,81.0,3,no
470,470,"(Yuca, Arveja grano seco)",68.0,53.968254,"['Arveja grano seco', 'Yuca']",0,0.0,0.0,inf,68.0,2,no
482,482,"(Yuca, Arveja grano seco, Frijol grano seco)",66.0,52.380952,"['Arveja grano seco', 'Frijol grano seco', 'Yuca']",0,0.0,0.0,inf,66.0,3,no
445,445,"(Maiz amarillo duro, Arveja grano verde, Arveja grano seco)",62.0,49.206349,"['Arveja grano seco', 'Arveja grano verde', 'Maiz amarillo duro']",0,0.0,0.0,inf,62.0,3,no
...,...,...,...,...,...,...,...,...,...,...,...,...
540,540,"(Zanahoria, Camote, Olluco, Maiz choclo, Maiz amilaceo, Yuca, Maiz amarillo duro, Cebada grano, Trigo, Arveja grano seco, Papa blanca, Frijol grano seco, Arveja grano verde)",1.0,0.793651,"['Arveja grano seco', 'Arveja grano verde', 'Camote', 'Cebada grano', 'Frijol grano seco', 'Maiz amarillo duro', 'Maiz amilaceo', 'Maiz choclo', 'Olluco', 'Papa blanca', 'Trigo', 'Yuca', 'Zanahoria']",0,0.0,0.0,inf,1.0,13,no
453,453,"(Paprika, Tomate, Cebolla cabeza roja, Aji, Frijol grano seco, Maiz amarillo duro, Yuca, Camote, Arroz cascara)",1.0,0.793651,"['Aji', 'Arroz cascara', 'Camote', 'Cebolla cabeza roja', 'Frijol grano seco', 'Maiz amarillo duro', 'Paprika', 'Tomate', 'Yuca']",0,0.0,0.0,inf,1.0,9,no
550,550,"(Zanahoria, Zapallo, Camote, Haba grano seco, Maiz choclo, Arveja grano seco, Papa blanca, Frijol grano seco, Maiz amilaceo, Yuca, Maiz amarillo duro, Cebada grano, Trigo, Arveja grano verde)",1.0,0.793651,"['Arveja grano seco', 'Arveja grano verde', 'Camote', 'Cebada grano', 'Frijol grano seco', 'Haba grano seco', 'Maiz amarillo duro', 'Maiz amilaceo', 'Maiz choclo', 'Papa blanca', 'Trigo', 'Yuca', 'Zanahoria', 'Zapallo']",0,0.0,0.0,inf,1.0,14,no
452,452,"(Cebolla cabeza roja, Tomate, Zanahoria, Ajo, Zapallo, Maiz amilaceo, Frijol grano seco, Papa blanca, Arveja grano seco, Arveja grano verde, Trigo, Cebada grano, Maiz amarillo duro, Maiz choclo, Haba grano seco, Yuca, Olluco, Papa color, Papa nativa)",1.0,0.793651,"['Ajo', 'Arveja grano seco', 'Arveja grano verde', 'Cebada grano', 'Cebolla cabeza roja', 'Frijol grano seco', 'Haba grano seco', 'Maiz amarillo duro', 'Maiz amilaceo', 'Maiz choclo', 'Olluco', 'Papa blanca', 'Papa color', 'Papa nativa', 'Tomate', 'Trigo', 'Yuca', 'Zanahoria', 'Zapallo']",0,0.0,0.0,inf,1.0,19,no
